Using the pymultinest as introduced here:

https://mattpitkin.github.io/samplers-demo/pages/samplers-samplers-everywhere/#PyMultiNest


In [1]:
import pymultinest

problem: [WinError 126] The specified module could not be found


SystemExit: 1

C:\Users\Joran\Anaconda3\envs\DD_DM\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import DirectDmTargets as dddm

In [ ]:
import wimprates as wr

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numericalunits as nu

In [2]:
from tqdm import tqdm

In [3]:
from scipy.integrate import quad as scipy_int

In [4]:
import pandas as pd

In [5]:
import scipy

In [6]:
import corner

In [7]:
import multiprocessing

# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

# Emcee #
<https://emcee.readthedocs.io/en/stable/tutorials/quickstart/>

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [8]:
use_SHM = dddm.SHM()
xe_events = dddm.GenSpectrum(50, 1e-45, use_SHM, dddm.detectors['Xe'])
xe_data = xe_events.get_data(poisson = True)
xe_data

NameError: name 'dddm' is not defined

In [ ]:
# priors = {
#     'log_mass':         {'range':[0.1,3],    'prior_type':'flat'},
#     'log_cross_secion': {'range':[-10, -6],    'prior_type':'flat'}, 
#     'density':          {'range':[0.001, 0.9], 'prior_type':'gauss', 'mean':0.4, 'std':0.1} ,
#     'v_0':              {'range':[80, 380],    'prior_type':'gauss', 'mean':230, 'std':30}, 
#     'v_esc':            {'range':[379, 709],   'prior_type':'gauss', 'mean':544, 'std':33}, 
#     'k':                {'range':[0.5, 3.5],   'prior_type':'flat'} 
# }

# for key in priors.keys():
#     param = priors[key]
#     if param['prior_type'] == 'flat':
#         param['param'] = param['range']
#         param['dist'] = lambda x: flat_prior(x)
#     elif param['prior_type'] == 'gauss':
#         param['param'] = param['mean'], param['std']
#         param['dist'] = lambda x: gaus_prior(x)
    

In [ ]:
# pos = 50 + np.random.randn(32, 1)
# nwalkers, ndim = pos.shape

# sampler = emcee.EnsembleSampler(nwalkers, ndim, 
#                 dddm.log_probability, args=(xe_data['bin_centers'],
#                                                             xe_data['counts'],
#                                                            'log_mass'))
# sampler.run_mcmc(pos, 50, progress=True);

In [ ]:
# fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
# samples = sampler.get_chain()
# labels = ["m", "b", "log(f)"]
# for i in range(ndim):
#     ax = axes[i]
#     ax.plot(samples[:, :, i], "k", alpha=0.3)
#     ax.set_xlim(0, len(samples))
#     ax.set_ylabel(labels[i])
#     ax.yaxis.set_label_coords(-0.1, 0.5)

# axes[-1].set_xlabel("step number");

In [ ]:
# fig = corner.corner(flat_samples, labels=labels, truths=[50, 1e-45]);

In [ ]:
# priors = {'log_cross_secion': {'range': [-46, -42], 'prior_type': 'flat'}}
# for key in priors.keys():
#     param = priors[key]
#     if param['prior_type'] == 'flat':
#         param['param'] = param['range']
#         param['dist'] = lambda x: flat_prior(x)
#     elif param['prior_type'] == 'gauss':
#         param['param'] = param['mean'], param['std']
#         param['dist'] = lambda x: gaus_prior(x)

In [ ]:
# def log_flat(x, x_name):
#     a, b = priors[x_name]['param']
#     try:
#         if a < x < b:
#             return 0
#         else:
#             return -np.inf
#     except ValueError:
#         result = np.zeros(len(x))
#         mask = (x > a) & (x < b)
#         result[~mask] = -np.inf
#         return result

In [ ]:
# def log_prior(x, x_name):
#     if priors[x_name]['prior_type'] == 'flat':
#         if 'log' in x_name:
#             return log_flat(np.log10(x), x_name)
#         else:
#             return log_flat(x, x_name)
#     elif priors[x_name]['prior_type'] == 'gauss':
#         return log_gauss(x, x_name)
#     else:
#         raise TypeError(f"unknown prior type '{priors[x_name]['prior_type']}', choose either gauss or flat")

In [ ]:
emcee.PTSampler

In [ ]:
!pip install emcee.PTSampler

In [ ]:
xe_events = dddm.GenSpectrum(50, 1e-45, use_SHM, dddm.detectors['Xe'])
xe_data = xe_events.get_data(poisson = False)

In [ ]:
from multiprocessing import Pool

In [ ]:
with Pool() as pool:
    nwalkers = 100
    steps   = 250
    pos = np.hstack(
        [50 + np.random.randn(nwalkers, 1) ,
        1e-45 + 1e-45 * np.random.randn(nwalkers, 1)])
    nwalkers, ndim = pos.shape

    sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                    dddm.log_probability, args=(xe_data['bin_centers'],
                                                                xe_data['counts'],
                                                               ['log_mass', 
                                                                'log_cross_section']), 
                                    pool=pool)
    sampler.run_mcmc(np.abs(pos), steps, progress=True);

In [ ]:
nwalkers = 100
steps   = 250
pos = np.hstack(
    [50 + np.random.randn(nwalkers, 1) ,
    1e-45 + 1e-45 * np.random.randn(nwalkers, 1)])
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                dddm.log_probability, args=(xe_data['bin_centers'],
                                                            xe_data['counts'],
                                                           ['log_mass', 
                                                            'log_cross_section']))
sampler.run_mcmc(np.abs(pos), steps, progress=True);

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["m", "sigma", "log(f)"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
# fig = corner.corner(flat_samples, labels=labels, truths=[50, 1e-45]);

In [ ]:
# tau = sampler.get_autocorr_time()
# print(tau)

In [ ]:
flat_samples = sampler.get_chain(discard=int(steps*0.2), thin=15, flat=True)
print(flat_samples.shape)

In [ ]:
fig = corner.corner(flat_samples, labels=labels, truths=[50, 1e-45]);

In [ ]:
# samples[:, 2] = np.exp(samples[:, 2])
# m_mcmc, b_mcmc, f_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
#                              zip(*np.percentile(samples, [16, 50, 84],
#                                                 axis=0)))

In [ ]:
nwalkers = 500
step = 1000
pos = np.hstack(
    [50   + 3*10 * np.random.rand(nwalkers, 1) ,
    1e-45 + 1e-45 * np.random.rand(nwalkers, 1),
    230   + 3*30 * np.random.rand(nwalkers, 1),
    544   + 3*33 * np.random.rand(nwalkers, 1),
    0.4   + 3*0.1 * np.random.rand(nwalkers, 1)   
    ])
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                dddm.log_probability, args=(xe_data['bin_centers'],
                                                            xe_data['counts'],
                                                           ['log_mass',
                                                            'log_cross_section', 
                                                            'v_0',
                                                            'v_esc', 
                                                            'density']))
sampler.run_mcmc(np.abs(pos), step, progress=True);

In [ ]:
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ['mass',
                                          'cross_section', 
                                                            'v_0',
                                                            'v_esc', 
                                                            'density']
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(discard=int(step*0.2), thin=15, flat=True)
print(flat_samples.shape)
fig = corner.corner(flat_samples, labels=['log_mass',
                                          'log_cross_section', 
                                                            'v_0',
                                                            'v_esc', 
                                                            'density'], 
                    truths=[50, 1e-45, 230, 544, 0.3]);

In [ ]:
samples_emcee = sampler.chain[:, int(step*0.2):, :].reshape((-1, ndim))

In [ ]:
np.shape(samples_emcee[4])

In [ ]:
for i in range(ndim):
    this_samp  = samples_emcee[:,i]
    print(f"{labels[i]}\tmean {np.mean(this_samp)}\tsig: {np.std(this_samp)}")

In [ ]:
print("Mean acceptance fraction: {0:.3f}"
                .format(np.mean(sampler.acceptance_fraction)))

In [ ]:
dddm.log_probability([7.89827908e+01, 4.55800899e-45, 3.78553277e+02, 1.56688729e+00,
 1.08146074e+00],
                    xe_data['bin_centers'],
                                                            xe_data['counts'],
                                                           ['log_mass',
                                                            'log_cross_section', 
                                                            'v_0',
                                                            'v_esc', 
                                                            'density'])

In [ ]:
dddm.log_probability([7.89827908e+01, 4.55800899e-45, 3.78553277e+02, 1.56688729e+00,
 1.08146074e+00],
                    xe_data['bin_centers'],
                                                            xe_data['counts'],
                                                           ['log_mass',
                                                            'log_cross_section', 
                                                            'v_0',
                                                            'v_esc', 
                                                            'density'])

In [ ]:
dddm.eval_spectrum([7.89827908e+01, 4.55800899e-45, 3.78553277e+02, 1.56688729e+00,
 1.08146074e+00], 
                                                           ['log_mass',
                                                            'log_cross_section', 
                                                            'v_0',
                                                            'v_esc', 
                                                            'density'])

In [ ]:
1.56688729e+00